# Lab | Working with APIs

Instructions  
Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

In [1]:
import pandas as pd
import requests
import json

In [ ]:
# Change <introduce your RapidAPI key here>

In [2]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}
    
    # Change <introduce your RapidAPI key here>
    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [ ]:
# Change <introduce your RapidAPI key here>

In [3]:
def flight_prices(departure, arrival, date):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    # Change <introduce your RapidAPI key here>
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

In [4]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

In [5]:
def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    return {date:flight_prices(origin, destination, date) for date in dates}


In [6]:
def lowest_price_data(origin, destination, start, end):
    data_flight = flight_days(origin, destination, start, end)
    
    
    df_flight_dates = pd.DataFrame()
    df_carriers = pd.DataFrame()
    df_places = pd.DataFrame()
    
    
    for day in data_flight.keys():
        df_flight_dates = df_flight_dates.append(pd.json_normalize(data_flight[day]["Quotes"]))
        df_carriers = df_carriers.append(pd.json_normalize(data_flight[day]['Carriers'])).drop_duplicates()
        df_places = df_places.append(pd.json_normalize(data_flight[day]['Places'])).drop_duplicates()
    

    price_cheap_flight = df_flight_dates.loc[(df_flight_dates['MinPrice'] == df_flight_dates['MinPrice'].min()),:]
    
    departure_date = price_cheap_flight['QuoteDateTime'].values[0]
    company_name = df_carriers.loc[(price_cheap_flight['OutboundLeg.CarrierIds'][0][0] == df_carriers['CarrierId']),'Name'].values[0]
    departure = df_places.loc[(df_places['PlaceId'] == price_cheap_flight['OutboundLeg.OriginId'][0]) ,'Name'].values[0]
    arrival = df_places.loc[(df_places['PlaceId'] == price_cheap_flight['OutboundLeg.DestinationId'][0]) ,'Name'].values[0]

    
    return print(f'Departure date: {departure_date[:10]} \nCompany name: {company_name} \nDeparture airport: {departure} \nArrival airport: {arrival}')



In [7]:
lowest_price_data('New York', 'London', '2020-12-20', '2020-12-22')

Departure date: 2020-12-01 
Company name: TAP Air Portugal 
Departure airport: New York Newark 
Arrival airport: London Heathrow
